<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/SN_web_lightmode.png" width="300">
</center>


<h1>Analysis of Global COVID-19 Pandemic Data</h1>

Estimated time needed: **90** minutes



## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [ ]:
# This lab requires 'httr' and 'rvest'packages, which are already pre-loaded into this lab environment.
# However, if you are working on your local RStudio, please uncomment the below codes and install the packages.

#install.packages("httr")
#install.packages("rvest")

In [5]:
library(httr)
library(rvest)

Note: if you can import above libraries, please use install.packages() to install them first.


## TASK 1: Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this 

URL https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [6]:
get_wiki_covid19_page <- function() {
    
base_url <- "https://en.wikipedia.org/w/index.php"
url_parameter <- "title=Template:COVID-19_testing_by_country"
full_url <- paste(base_url, "?", url_parameter, sep = "")
response <- GET(url = full_url)
return(response)
}



Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [7]:
# Call the get_wiki_covid19_page function and print the response
get_wiki_covid19_page()

Response [https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country]
  Date: 2024-04-04 16:39
  Status: 200
  Content-Type: text/html; charset=UTF-8
  Size: 449 kB
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-fea...
<head>
<meta charset="UTF-8">
<title>Template:COVID-19 testing by country - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-heade...
"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["...
"CS1 uses Russian-language script (ru)","CS1 Russian-language sources (ru)","...
,"CS1 Lithuanian-language sources (lt)","CS1 Malagasy-language sources (mg)",...
"wgRelevantArticleId":63303421,"wgIsProbablyEditable":false,"wgRelevantPageIs...
...

## TASK 2: Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [9]:
root_html <- read_html(get_wiki_covid19_page())
root_node <- xml_root(root_html)
root_node

{html_document}
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body class="skin-vector skin-vector-search-vue mediawiki ltr sitedir-ltr ...

Get the tables in the HTML root node using `html_nodes` function.


In [10]:
# Get the table node from the root html node
table_node<-html_nodes(root_node, "table")
table_node

{xml_nodeset (4)}
[1] <table class="box-Update plainlinks ombox ombox-content ambox-Update" rol ...
[2] <table class="wikitable plainrowheaders sortable collapsible autocollapse ...
[3] <table class="plainlinks ombox mbox-small ombox-notice" role="presentatio ...
[4] <table class="wikitable mw-templatedata-doc-params">\n<caption><p class=" ...

Read the specific table from the multiple tables in the `table_node` using the `html_table` function and convert it into dataframe using `as.data.frame`

_Hint:- Please read the `table_node` with index 2(ex:- table_node[2])._


In [12]:
# Read the table node and convert it into a data frame, and print the data frame for review
CovidDF<-as.data.frame(html_table(table_node[2]))
head(CovidDF)

,Country.or.region,Date.a.,Tested,Units.b.,Confirmed.cases.,Confirmed..tested..,Tested..population..,Confirmed..population..,Ref.
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Afghanistan,17 Dec 2020,"154,767",samples,"49,621",32.1,0.40,0.13,[1]
2,Albania,18 Feb 2021,"428,654",samples,"96,838",22.6,15.0,3.4,[2]
3,Algeria,2 Nov 2020,"230,553",samples,"58,574",25.4,0.53,0.13,[3][4]
4,Andorra,23 Feb 2022,"300,307",samples,"37,958",12.6,387,49.0,[5]
5,Angola,2 Feb 2021,"399,228",samples,"20,981",5.3,1.3,0.067,[6]
6,Antigua and Barbuda,6 Mar 2021,"15,268",samples,832,5.4,15.9,0.86,[7]


## TASK 3: Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame


In [13]:
# Print the summary of the data frame
summary(CovidDF)

 Country.or.region    Date.a.             Tested            Units.b.        
 Length:173         Length:173         Length:173         Length:173        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 Confirmed.cases.   Confirmed..tested.. Tested..population..
 Length:173         Length:173          Length:173          
 Class :character   Class :character    Class :character    
 Mode  :character   Mode  :character    Mode  :character    
 Confirmed..population..     Ref.          
 Length:173              Length:173        
 Class :character        Class :character  
 Mode  :character        Mode  :character  

As you can see from the summary, the columns names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`. 

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [15]:
preprocess_covid_data_frame <- function(data_frame) {
    
    shape <- dim(data_frame)

    # Remove the World row
    data_frame<-data_frame[!(data_frame$`Country.or.region`=="World"),]
    # Remove the last row
    data_frame <- data_frame[1:172, ]
    
    # We dont need the Units and Ref columns, so can be removed
    data_frame["Ref."] <- NULL
    data_frame["Units.b."] <- NULL
    
    # Renaming the columns
    names(data_frame) <- c("country", "date", "tested", "confirmed", "confirmed.tested.ratio", "tested.population.ratio", "confirmed.population.ratio")
    
    # Convert column data types
    data_frame$country <- as.factor(data_frame$country)
    data_frame$date <- as.factor(data_frame$date)
    data_frame$tested <- as.numeric(gsub(",","",data_frame$tested))
    data_frame$confirmed <- as.numeric(gsub(",","",data_frame$confirmed))
    data_frame$'confirmed.tested.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.tested.ratio`))
    data_frame$'tested.population.ratio' <- as.numeric(gsub(",","",data_frame$`tested.population.ratio`))
    data_frame$'confirmed.population.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.population.ratio`))
    
    return(data_frame)
}


Call the `preprocess_covid_data_frame` function


In [18]:
# call `preprocess_covid_data_frame` function and assign it to a new data frame
CovidDF.New = preprocess_covid_data_frame(CovidDF)
head(CovidDF.New)

,country,date,tested,confirmed,confirmed.tested.ratio,tested.population.ratio,confirmed.population.ratio
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,17 Dec 2020,154767,49621,32.1,0.40,0.130
2,Albania,18 Feb 2021,428654,96838,22.6,15.00,3.400
3,Algeria,2 Nov 2020,230553,58574,25.4,0.53,0.130
4,Andorra,23 Feb 2022,300307,37958,12.6,387.00,49.000
5,Angola,2 Feb 2021,399228,20981,5.3,1.30,0.067
6,Antigua and Barbuda,6 Mar 2021,15268,832,5.4,15.90,0.860


Get the summary of the processed data frame again


In [19]:
# Print the summary of the processed data frame again
summary(CovidDF.New)

                country             date         tested         
 Afghanistan        :  1   2 Feb 2023 :  6   Min.   :     3880  
 Albania            :  1   1 Feb 2023 :  4   1st Qu.:   512037  
 Algeria            :  1   31 Jan 2023:  4   Median :  3029859  
 Andorra            :  1   1 Mar 2021 :  3   Mean   : 31377219  
 Angola             :  1   23 Jul 2021:  3   3rd Qu.: 12386725  
 Antigua and Barbuda:  1   29 Jan 2023:  3   Max.   :929349291  
 (Other)            :166   (Other)    :149                      
   confirmed        confirmed.tested.ratio tested.population.ratio
 Min.   :       0   Min.   : 0.00          Min.   :   0.006       
 1st Qu.:   37839   1st Qu.: 5.00          1st Qu.:   9.475       
 Median :  281196   Median :10.05          Median :  46.950       
 Mean   : 2508340   Mean   :11.25          Mean   : 175.504       
 3rd Qu.: 1278105   3rd Qu.:15.25          3rd Qu.: 156.500       
 Max.   :90749469   Max.   :46.80          Max.   :3223.000       
           

After pre-processing, you can see the columns and columns names are simplified, and columns types are converted into correct types.


The data frame has following columns:

- **country** - The name of the country
- **date** - Reported date
- **tested** - Total tested cases by the reported date
- **confirmed** - Total confirmed cases by the reported date
- **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
- **tested.population.ratio** - The ratio of tested cases to the population of the country
- **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file. 


In [21]:
# Export the data frame to a csv file
write.csv(CovidDF.New, file = 'Covid.csv')

Note for IBM Waston Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [22]:
# Get working directory
wd <- getwd()
# Get exported 
file_path <- paste(wd, sep="", "/covid.csv")
# File path
print(file_path)
file.exists(file_path)

[1] "/resources/labs/authoride/IBMSkillsNetwork+RP0101EN/v2/M5_Final/covid.csv"


[1] FALSE

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [ ]:
## Download a sample csv file
# covid_csv_file <- download.file("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/dataset/covid.csv", destfile="covid.csv")
# covid_data_frame_csv <- read.csv("covid.csv", header=TRUE, sep=",")

## TASK 4: Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [36]:
# Read covid_data_frame_csv from the csv file
covid_data_frame_csv = read.csv('Covid.csv')

# Get the 5th to 10th rows, with two "country" "confirmed" columns
covid_data_frame_csv[5:10,c('country', 'confirmed')]

,country,confirmed
,<fct>,<int>
5,Angola,20981
6,Antigua and Barbuda,832
7,Argentina,9060495
8,Armenia,422963
9,Australia,10112229
10,Austria,5789991


## TASK 5: Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [37]:
# Get the total confirmed cases worldwide
sum(covid_data_frame_csv$confirmed)

# Get the total tested cases worldwide
sum(covid_data_frame_csv$tested)

# Get the positive ratio (confirmed / tested)
sum(covid_data_frame_csv$confirmed)/sum(covid_data_frame_csv$tested)


[1] 431434555

[1] 5396881644

[1] 0.07994145

## TASK 6: Get a country list which reported their testing data 

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [38]:
# Get the `country` column
covid_data_frame_csv$country

# Check its class (should be Factor)
class(covid_data_frame_csv$country)

[1] Afghanistan            Albania                Algeria               
  [4] Andorra                Angola                 Antigua and Barbuda   
  [7] Argentina              Armenia                Australia             
 [10] Austria                Azerbaijan             Bahamas               
 [13] Bahrain                Bangladesh             Barbados              
 [16] Belarus                Belgium                Belize                
 [19] Benin                  Bhutan                 Bolivia               
 [22] Bosnia and Herzegovina Botswana               Brazil                
 [25] Brunei                 Bulgaria               Burkina Faso          
 [28] Burundi                Cambodia               Cameroon              
 [31] Canada                 Chad                   Chile                 
 [34] China[c]               Colombia               Costa Rica            
 [37] Croatia                Cuba                   Cyprus[d]             
 [40] Czechia                Denmark[e]             Djibouti              
 [43] Dominica               Dominican Republic     DR Congo              
 [46] Ecuador                Egypt                  El Salvador           
 [49] Equatorial Guinea      Estonia                Eswatini              
 [52] Ethiopia               Faroe Islands          Fiji                  
 [55] Finland                France[f][g]           Gabon                 
 [58] Gambia                 Georgia[h]             Germany               
 [61] Ghana                  Greece                 Greenland             
 [64] Grenada                Guatemala              Guinea                
 [67] Guinea-Bissau          Guyana                 Haiti                 
 [70] Honduras               Hungary                Iceland               
 [73] India                  Indonesia              Iran                  
 [76] Iraq                   Ireland                Israel                
 [79] Italy                  Ivory Coast            Jamaica               
 [82] Japan                  Jordan                 Kazakhstan            
 [85] Kenya                  Kosovo                 Kuwait                
 [88] Kyrgyzstan             Laos                   Latvia                
 [91] Lebanon                Lesotho                Liberia               
 [94] Libya                  Lithuania              Luxembourg[i]         
 [97] Madagascar             Malawi                 Malaysia              
[100] Maldives               Mali                   Malta                 
[103] Mauritania             Mauritius              Mexico                
[106] Moldova[j]             Mongolia               Montenegro            
[109] Morocco                Mozambique             Myanmar               
[112] Namibia                Nepal                  Netherlands           
[115] New Caledonia          New Zealand            Niger                 
[118] Nigeria                North Korea            North Macedonia       
[121] Northern Cyprus[k]     Norway                 Oman                  
[124] Pakistan               Palestine              Panama                
[127] Papua New Guinea       Paraguay               Peru                  
[130] Philippines            Poland                 Portugal              
[133] Qatar                  Romania                Russia                
[136] Rwanda                 Saint Kitts and Nevis  Saint Lucia           
[139] Saint Vincent          San Marino             Saudi Arabia          
[142] Senegal                Serbia                 Singapore             
[145] Slovakia               Slovenia               South Africa          
[148] South Korea            South Sudan            Spain                 
[151] Sri Lanka              Sudan                  Sweden                
[154] Switzerland[l]         Taiwan[m]              Tanzania              
[157] Thailand               Togo                   Trinidad and Tobago   
[160] Tunisia              

[1] "factor"

In [39]:
# Conver the country column into character so that you can easily sort them
covid_data_frame_csv$country = as.character(covid_data_frame_csv$country)
class(covid_data_frame_csv$country)

[1] "character"

In [40]:
# Sort the countries AtoZ
sort(covid_data_frame_csv$country)

[1] "Afghanistan"            "Albania"                "Algeria"               
  [4] "Andorra"                "Angola"                 "Antigua and Barbuda"   
  [7] "Argentina"              "Armenia"                "Australia"             
 [10] "Austria"                "Azerbaijan"             "Bahamas"               
 [13] "Bahrain"                "Bangladesh"             "Barbados"              
 [16] "Belarus"                "Belgium"                "Belize"                
 [19] "Benin"                  "Bhutan"                 "Bolivia"               
 [22] "Bosnia and Herzegovina" "Botswana"               "Brazil"                
 [25] "Brunei"                 "Bulgaria"               "Burkina Faso"          
 [28] "Burundi"                "Cambodia"               "Cameroon"              
 [31] "Canada"                 "Chad"                   "Chile"                 
 [34] "China[c]"               "Colombia"               "Costa Rica"            
 [37] "Croatia"                "Cuba"                   "Cyprus[d]"             
 [40] "Czechia"                "Denmark[e]"             "Djibouti"              
 [43] "Dominica"               "Dominican Republic"     "DR Congo"              
 [46] "Ecuador"                "Egypt"                  "El Salvador"           
 [49] "Equatorial Guinea"      "Estonia"                "Eswatini"              
 [52] "Ethiopia"               "Faroe Islands"          "Fiji"                  
 [55] "Finland"                "France[f][g]"           "Gabon"                 
 [58] "Gambia"                 "Georgia[h]"             "Germany"               
 [61] "Ghana"                  "Greece"                 "Greenland"             
 [64] "Grenada"                "Guatemala"              "Guinea"                
 [67] "Guinea-Bissau"          "Guyana"                 "Haiti"                 
 [70] "Honduras"               "Hungary"                "Iceland"               
 [73] "India"                  "Indonesia"              "Iran"                  
 [76] "Iraq"                   "Ireland"                "Israel"                
 [79] "Italy"                  "Ivory Coast"            "Jamaica"               
 [82] "Japan"                  "Jordan"                 "Kazakhstan"            
 [85] "Kenya"                  "Kosovo"                 "Kuwait"                
 [88] "Kyrgyzstan"             "Laos"                   "Latvia"                
 [91] "Lebanon"                "Lesotho"                "Liberia"               
 [94] "Libya"                  "Lithuania"              "Luxembourg[i]"         
 [97] "Madagascar"             "Malawi"                 "Malaysia"              
[100] "Maldives"               "Mali"                   "Malta"                 
[103] "Mauritania"             "Mauritius"              "Mexico"                
[106] "Moldova[j]"             "Mongolia"               "Montenegro"            
[109] "Morocco"                "Mozambique"             "Myanmar"               
[112] "Namibia"                "Nepal"                  "Netherlands"           
[115] "New Caledonia"          "New Zealand"            "Niger"                 
[118] "Nigeria"                "North Korea"            "North Macedonia"       
[121] "Northern Cyprus[k]"     "Norway"                 "Oman"                  
[124] "Pakistan"               "Palestine"              "Panama"                
[127] "Papua New Guinea"       "Paraguay"               "Peru"                  
[130] "Philippines"            "Poland"                 "Portugal"              
[133] "Qatar"                  "Romania"                "Russia"                
[136] "Rwanda"                 "Saint Kitts and Nevis"  "Saint Lucia"           
[139] "Saint Vincent"          "San Marino"             "Saudi Arabia"          
[142] "Senegal"                "Serbia"                 "Singapore"             
[145] "Slovakia"               "Slovenia"               "South Africa"          
[148] "South Korea"            "S

In [41]:
# Sort the countries ZtoA
sort(covid_data_frame_csv$country, decreasing = TRUE)

# Print the sorted ZtoA list
print(sort(covid_data_frame_csv$country, decreasing = TRUE))


[1] "Zimbabwe"               "Zambia"                 "Vietnam"               
  [4] "Venezuela"              "Uzbekistan"             "Uruguay"               
  [7] "United States"          "United Kingdom"         "United Arab Emirates"  
 [10] "Ukraine"                "Uganda"                 "Turkey"                
 [13] "Tunisia"                "Trinidad and Tobago"    "Togo"                  
 [16] "Thailand"               "Tanzania"               "Taiwan[m]"             
 [19] "Switzerland[l]"         "Sweden"                 "Sudan"                 
 [22] "Sri Lanka"              "Spain"                  "South Sudan"           
 [25] "South Korea"            "South Africa"           "Slovenia"              
 [28] "Slovakia"               "Singapore"              "Serbia"                
 [31] "Senegal"                "Saudi Arabia"           "San Marino"            
 [34] "Saint Vincent"          "Saint Lucia"            "Saint Kitts and Nevis" 
 [37] "Rwanda"                 "Russia"                 "Romania"               
 [40] "Qatar"                  "Portugal"               "Poland"                
 [43] "Philippines"            "Peru"                   "Paraguay"              
 [46] "Papua New Guinea"       "Panama"                 "Palestine"             
 [49] "Pakistan"               "Oman"                   "Norway"                
 [52] "Northern Cyprus[k]"     "North Macedonia"        "North Korea"           
 [55] "Nigeria"                "Niger"                  "New Zealand"           
 [58] "New Caledonia"          "Netherlands"            "Nepal"                 
 [61] "Namibia"                "Myanmar"                "Mozambique"            
 [64] "Morocco"                "Montenegro"             "Mongolia"              
 [67] "Moldova[j]"             "Mexico"                 "Mauritius"             
 [70] "Mauritania"             "Malta"                  "Mali"                  
 [73] "Maldives"               "Malaysia"               "Malawi"                
 [76] "Madagascar"             "Luxembourg[i]"          "Lithuania"             
 [79] "Libya"                  "Liberia"                "Lesotho"               
 [82] "Lebanon"                "Latvia"                 "Laos"                  
 [85] "Kyrgyzstan"             "Kuwait"                 "Kosovo"                
 [88] "Kenya"                  "Kazakhstan"             "Jordan"                
 [91] "Japan"                  "Jamaica"                "Ivory Coast"           
 [94] "Italy"                  "Israel"                 "Ireland"               
 [97] "Iraq"                   "Iran"                   "Indonesia"             
[100] "India"                  "Iceland"                "Hungary"               
[103] "Honduras"               "Haiti"                  "Guyana"                
[106] "Guinea-Bissau"          "Guinea"                 "Guatemala"             
[109] "Grenada"                "Greenland"              "Greece"                
[112] "Ghana"                  "Germany"                "Georgia[h]"            
[115] "Gambia"                 "Gabon"                  "France[f][g]"          
[118] "Finland"                "Fiji"                   "Faroe Islands"         
[121] "Ethiopia"               "Eswatini"               "Estonia"               
[124] "Equatorial Guinea"      "El Salvador"            "Egypt"                 
[127] "Ecuador"                "DR Congo"               "Dominican Republic"    
[130] "Dominica"               "Djibouti"               "Denmark[e]"            
[133] "Czechia"                "Cyprus[d]"              "Cuba"                  
[136] "Croatia"                "Costa Rica"             "Colombia"              
[139] "China[c]"               "Chile"                  "Chad"                  
[142] "Canada"                 "Cameroon"               "Cambodia"              
[145] "Burundi"                "Burkina Faso"           "Bulgaria"              
[148] "Brunei"                 "B

  [1] "Zimbabwe"               "Zambia"                 "Vietnam"               
  [4] "Venezuela"              "Uzbekistan"             "Uruguay"               
  [7] "United States"          "United Kingdom"         "United Arab Emirates"  
 [10] "Ukraine"                "Uganda"                 "Turkey"                
 [13] "Tunisia"                "Trinidad and Tobago"    "Togo"                  
 [16] "Thailand"               "Tanzania"               "Taiwan[m]"             
 [19] "Switzerland[l]"         "Sweden"                 "Sudan"                 
 [22] "Sri Lanka"              "Spain"                  "South Sudan"           
 [25] "South Korea"            "South Africa"           "Slovenia"              
 [28] "Slovakia"               "Singapore"              "Serbia"                
 [31] "Senegal"                "Saudi Arabia"           "San Marino"            
 [34] "Saint Vincent"          "Saint Lucia"            "Saint Kitts and Nevis" 
 [37] "Rwanda"              

## TASK 7: Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires start with `United`


In [42]:
# Use a regular expression `United.+` to find matches
Index = grep('United.+', covid_data_frame_csv$country)

# Print the matched country names
for (i in Index) {print(covid_data_frame_csv$country[i])}

[1] "United Arab Emirates"
[1] "United Kingdom"
[1] "United States"


## TASK 8: Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [46]:
# Select a subset (should be only one row) of data frame based on a selected country name and columns
print(covid_data_frame_csv[35,c(2, 5, 8)])


# Select a subset (should be only one row) of data frame based on a selected country name and columns
print(covid_data_frame_csv[88,c(2, 5, 8)])

    country confirmed confirmed.population.ratio
35 Colombia   6314769                       13.1
      country confirmed confirmed.population.ratio
88 Kyrgyzstan     85253                        1.3


## TASK 9: Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [49]:
# Use if-else statement
if (covid_data_frame_csv[35,8] > covid_data_frame_csv[88,8]) {
    print('Columbia Has The Higher Ratio Of Confirmed Cases To Population')
 } else {
    print('Kyrgzstan Has The Higher Ratio Of Confirmed Cases To Population')
 }


[1] "Columbia Has The Higher Ratio Of Confirmed Cases To Population"


## TASK 10: Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [58]:
# Get a subset of any countries with `confirmed.population.ratio` less than the threshold
CovidSubset = subset(covid_data_frame_csv, subset = confirmed.population.ratio < 1)
print(CovidSubset[,c(2,8)])

                country confirmed.population.ratio
1           Afghanistan                    0.13000
3               Algeria                    0.13000
5                Angola                    0.06700
6   Antigua and Barbuda                    0.86000
14           Bangladesh                    0.70000
19                Benin                    0.06700
25               Brunei                    0.07400
27         Burkina Faso                    0.05800
28              Burundi                    0.00740
29             Cambodia                    0.48000
30             Cameroon                    0.12000
32                 Chad                    0.02900
34             China[c]                    0.00610
45             DR Congo                    0.02900
47                Egypt                    0.28000
52             Ethiopia                    0.24000
57                Gabon                    0.08200
58               Gambia                    0.21000
61                Ghana        